In [1]:
#1. Create Spark Session

# Testing pyspark Intallation
import findspark
findspark.init('C:\Spark')
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

spark

c:\users\bmi_cims\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\bmi_cims\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
c:\users\bmi_cims\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
wordsList = ['cat', 'elephant', 'rat', 'rat']
wordsRDD = spark.sparkContext.parallelize(wordsList, 4)
# Print out the type of wordsRDD
print(type(wordsRDD))

<class 'pyspark.rdd.RDD'>


In [4]:
def makePlural(word):
    return word + 's'

In [5]:
print(makePlural('cat'))

cats


In [6]:
pluralRDD = wordsRDD.map(makePlural)
print(pluralRDD.collect())

['cats', 'elephants', 'rats', 'rats']


In [7]:
pluralLambdaRDD = wordsRDD.map(lambda animal : animal + 's')
print(pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats']


In [8]:
wordPairs = wordsRDD.map(lambda animal : (animal,1))
print(wordPairs.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1)]


In [10]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

cat: [1]
elephant: [1]
rat: [1, 1]


In [13]:
wordCountsGrouped = wordsGrouped.map(lambda (key,value) : (key, sum(value)))
print(wordCountsGrouped.collect())

SyntaxError: invalid syntax (<ipython-input-13-a07971e94193>, line 1)

In [14]:
wordCounts = wordPairs.reduceByKey(lambda val1,val2 : val1+val2)
print(wordCounts.collect())

[('cat', 1), ('elephant', 1), ('rat', 2)]


In [15]:
wordCountsCollected = (wordsRDD.map(lambda animal : (animal,1)).reduceByKey(lambda val1,val2 : val1+val2).collect())
print(wordCountsCollected)

[('cat', 1), ('elephant', 1), ('rat', 2)]


In [16]:
uniqueWords = wordsRDD.distinct().count()
print(uniqueWords)

3


In [19]:
from operator import add
totalCount = (wordCounts.map(lambda(key,value) : value).reduce(add))
average = totalCount / float(uniqueWords)
print(totalCount)
print(round(average, 2))

SyntaxError: invalid syntax (<ipython-input-19-06e88203b08d>, line 2)

In [ ]:
from pyspark.sql.functions import *

In [ ]:
initDF = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

In [ ]:
initDF.isStreaming

In [ ]:
resultDF = initDF.withColumn("result", col("value") + lit(1))

In [ ]:
resultDF.writeStream.outputMode("append").option("truncate",False).format("console").start().awaitTermination()

In [ ]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

# Generate running word count
wordCounts = words.groupBy("word").count()

In [ ]:

# Define host and port number to Listen.
host = "127.0.0.1"
port = "9999"

# Create Streaming DataFrame by reading data from socket.
val initDF = (spark.readStream.format("socket").option("host", host).option("port", port).load())

# Check if DataFrame is streaming or Not.
println("Streaming DataFrame : " + initDF.isStreaming)

In [ ]:
# here we are trying to understsnd how rdds works manually

rdd1 = spark.sparkContext.parallelize(['santhosh'])
rdd2 = spark.sparkContext.parallelize(['anand'])
rdd3 = spark.sparkContext.parallelize(['Prashanth'])

rdds = [rdd1,rdd2,rdd3]

for rdd in rdds:
    print("Length of the chars in the name:",rdd.map(lambda x : len(x)).collect())

In [ ]:
# Streaming the rdds  using 'queueStream'

from pyspark.streaming import StreamingContext
import time

ssc = StreamingContext(spark.sparkContext, 0.5)  # batch duration = 0.5 seconds

inputStream = ssc.queueStream(rdds)

mappedStream = inputStream.map(lambda x : len(x))
mappedStream.pprint()

# Here streaming will run for 6 seconds as there is sleep of 6 seconds before streaming stops
ssc.start()
time.sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True)